# Summary of Qiskit Optimization 0.2.0

Takashi Imamichi (imamichi@jp.ibm.com), IBM Quantum.

Qiskit Optimization 0.2.0 introduces the following new features:

- Translators between `QuadraticProgram` and other objects such as an optimization model and an Ising Hamiltonian (`qiskit_optimization.translators`)
- Support of gurobipy (`GurobiSolver`, `from_gurobipy`, and `to_gurobipy`)
- Support of indicator constraints at `from_docplex_mp`
- Support of solver parameter of `CplexSolver`

It also includes various bug fixes and code cleanups. See the release note for details.

In [1]:
%load_ext autoreload
%autoreload 2

## Translator

Qiskit Optimization 0.2.0 allows us to load and save an optimization model by third party libraries. It used to support only Docplex and starts to support Gurobi.

In [2]:
# docplex model
from docplex.mp.model import Model
docplex_model = Model('docplex')
x = docplex_model.binary_var('x')
y = docplex_model.integer_var(-1, 4, 'y')
docplex_model.maximize(x * y)
docplex_model.add_constraint(x <= y)
print(docplex_model.export_as_lp_string())
print(docplex_model.solve())

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: docplex

Maximize
 obj: [ 2 x*y ]/2
Subject To
 c1: x - y <= 0

Bounds
 0 <= x <= 1
 -1 <= y <= 4

Binaries
 x

Generals
 y
End

solution for: docplex
objective: 4
x=1
y=4



In [3]:
# gurobi model
import gurobipy as gp
gurobipy_model = gp.Model('gurobi')
x = gurobipy_model.addVar(vtype=gp.GRB.BINARY, name="x")
y = gurobipy_model.addVar(vtype=gp.GRB.INTEGER, lb=-1, ub=4, name="y")
gurobipy_model.setObjective(x * y, gp.GRB.MAXIMIZE)
gurobipy_model.addConstr(x - y <= 0)
gurobipy_model.Params.OutputFlag = 0
gurobipy_model.optimize()
gurobipy_model.display()
print('objective:', gurobipy_model.ObjVal)
print('x:', gurobipy_model.X)

Restricted license - for non-production use only - expires 2022-01-13
Maximize
   <gurobi.QuadExpr: 0.0 + [ x * y ]>
Subject To
   R0 : <gurobi.LinExpr: x + -1.0 y> <= 0.0
Bounds
   -1.0 <= y <= 4.0
Binaries
   ['x']
General Integers
   ['y']
objective: 4.0
x: [1.0, 4.0]


We can generate `QuadraticProgram` object from both Docplex and Gurobi models.

In [4]:
from qiskit_optimization.translators import from_docplex_mp, from_gurobipy
qp = from_docplex_mp(docplex_model)
print('from docpblex')
print(qp.export_as_lp_string())
print('-------------')
print('from gurobipy')
qp2 = from_gurobipy(gurobipy_model)
print(qp2.export_as_lp_string())

from docpblex
\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: docplex

Maximize
 obj: [ 2 x*y ]/2
Subject To
 c0: x - y <= 0

Bounds
 0 <= x <= 1
 -1 <= y <= 4

Binaries
 x

Generals
 y
End

-------------
from gurobipy
\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: gurobi

Maximize
 obj: [ 2 x*y ]/2
Subject To
 R0: x - y <= 0

Bounds
 0 <= x <= 1
 -1 <= y <= 4

Binaries
 x

Generals
 y
End



We can generate a Docplex model and a Gurobi model from `QuadraticProgram` as well.

In [5]:
from qiskit_optimization.translators import to_gurobipy, to_docplex_mp
gmod = to_gurobipy(from_docplex_mp(docplex_model))
print('convert docplex to gurobipy')
gmod.display()

dmod = to_docplex_mp(from_gurobipy(gurobipy_model))
print('\nconvert gurobi to docplex')
dmod.prettyprint()

convert docplex to gurobipy
Maximize
   <gurobi.QuadExpr: 0.0 + [ x * y ]>
Subject To
   c0 : <gurobi.LinExpr: x + -1.0 y> <= 0.0
Bounds
   -1.0 <= y <= 4.0
Binaries
   ['x']
General Integers
   ['y']

convert gurobi to docplex
// This file has been generated by DOcplex
// model name is: gurobi
// single vars section
dvar bool x;
dvar int y;

maximize
 [ x*y ];
 
subject to {
 R0:
  x - y <= 0;

}


## GurobiSolver and CplexSolver

Qiskit optimization supports the solvers of CPLEX and Gurobi.

In [6]:
from qiskit_optimization.algorithms import CplexOptimizer, GurobiOptimizer
print('cplex', CplexOptimizer().solve(qp))
print()
print('gurobi', GurobiOptimizer().solve(qp))

cplex optimal function value: 4.0
optimal value: [1. 4.]
status: SUCCESS

gurobi optimal function value: 4.0
optimal value: [1. 4.]
status: SUCCESS


We can set the solver parameter of CPLEX as follows. See [Parameters of CPLEX](https://www.ibm.com/docs/en/icos/20.1.0?topic=cplex-parameters) for details of the parameters.

In [7]:
CplexOptimizer(disp=True, cplex_parameters={'threads': 1, 'timelimit': 0.1}).solve(qp)

Version identifier: 20.1.0.0 | 2020-11-10 | 9bedb6d68
CPXPARAM_Read_DataCheck                          1
CPXPARAM_Threads                                 1
CPXPARAM_TimeLimit                               0.10000000000000001
Found incumbent of value 0.000000 after 0.00 sec. (0.00 ticks)
Found incumbent of value 4.000000 after 0.00 sec. (0.00 ticks)

Root node processing (before b&c):
  Real time             =    0.00 sec. (0.00 ticks)
Sequential b&c:
  Real time             =    0.00 sec. (0.00 ticks)
                          ------------
Total (root+branch&cut) =    0.00 sec. (0.00 ticks)


optimal function value: 4.0
optimal value: [1. 4.]
status: SUCCESS

FYI: We can solve the same problem by QAOA and VQE (this is not changed).

In [8]:
from qiskit_optimization.algorithms import MinimumEigenOptimizer

from qiskit import Aer
from qiskit.utils import QuantumInstance
from qiskit.algorithms import QAOA
from qiskit.algorithms.optimizers import COBYLA

qins = QuantumInstance(backend=Aer.get_backend('aer_simulator'), shots=1000)
meo = MinimumEigenOptimizer(QAOA(COBYLA(maxiter=100), quantum_instance=qins))
result = meo.solve(qp)
print(result)
for sample in result.samples:
    print(sample)

optimal function value: 4.0
optimal value: [1. 4.]
status: SUCCESS
SolutionSample(x=array([1., 4.]), fval=4.0, probability=0.007999999999999998, status=<OptimizationResultStatus.SUCCESS: 0>)
SolutionSample(x=array([1., 3.]), fval=3.0, probability=0.157, status=<OptimizationResultStatus.SUCCESS: 0>)
SolutionSample(x=array([1., 2.]), fval=2.0, probability=0.12, status=<OptimizationResultStatus.SUCCESS: 0>)
SolutionSample(x=array([1., 1.]), fval=1.0, probability=0.054, status=<OptimizationResultStatus.SUCCESS: 0>)
SolutionSample(x=array([0., 1.]), fval=0.0, probability=0.21900000000000008, status=<OptimizationResultStatus.SUCCESS: 0>)
SolutionSample(x=array([0., 2.]), fval=0.0, probability=0.018000000000000002, status=<OptimizationResultStatus.SUCCESS: 0>)
SolutionSample(x=array([0., 3.]), fval=0.0, probability=0.14600000000000002, status=<OptimizationResultStatus.SUCCESS: 0>)
SolutionSample(x=array([0., 4.]), fval=0.0, probability=0.09300000000000001, status=<OptimizationResultStatus.SUC

## Indicator constraints

`from_docplex_mp` supports indicator constraints, e.g., `u = 0 => x + y <= z` (u: binary variable) when we convert a Docplex model into `QuadraticProgram`. It converts indicator constraints into linear constraints using the big-M formulation.

In [9]:
ind_mod = Model('docplex')
x = ind_mod.binary_var('x')
y = ind_mod.integer_var(-1, 2, 'y')
z = ind_mod.integer_var(-1, 2, 'z')
ind_mod.maximize(3 * x + y - z)
ind_mod.add_indicator(x, y >= z, 1)
print(ind_mod.export_as_lp_string())
print(ind_mod.solve())

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: docplex

Maximize
 obj: 3 x + y - z
Subject To
 lc1: x = 1 -> y - z >= 0

Bounds
 0 <= x <= 1
 -1 <= y <= 2
 -1 <= z <= 2

Binaries
 x

Generals
 y z
End

solution for: docplex
objective: 6
x=1
y=2
z=-1



In [10]:
qp = from_docplex_mp(ind_mod)
print(qp.export_as_lp_string())
result = meo.solve(qp)
print(result)
for sample in result.samples:
    print(sample)

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: docplex

Maximize
 obj: 3 x + y - z
Subject To
 c0: - 3 x + y - z >= -3

Bounds
 0 <= x <= 1
 -1 <= y <= 2
 -1 <= z <= 2

Binaries
 x

Generals
 y z
End

optimal function value: 6.0
optimal value: [ 1.  2. -1.]
status: SUCCESS
SolutionSample(x=array([ 1.,  2., -1.]), fval=6.0, probability=0.048, status=<OptimizationResultStatus.SUCCESS: 0>)
SolutionSample(x=array([ 1.,  1., -1.]), fval=5.0, probability=0.033, status=<OptimizationResultStatus.SUCCESS: 0>)
SolutionSample(x=array([1., 2., 0.]), fval=5.0, probability=0.036, status=<OptimizationResultStatus.SUCCESS: 0>)
SolutionSample(x=array([ 1.,  0., -1.]), fval=4.0, probability=0.039, status=<OptimizationResultStatus.SUCCESS: 0>)
SolutionSample(x=array([1., 1., 0.]), fval=4.0, probability=0.04, status=<OptimizationResultStatus.SUCCESS: 0>)
SolutionSample(x=array([1., 2., 1.]), fval=4.0, probability=0.049999999999999996, status=<OptimizationResultStatus.SUCCES

## Deprecation

`QuadraticProgram.from_docplex` and `QuadraticProgram.to_docplex` are deprecated. We need to use `from_docplex_mp` and `to_docplex_mp` instead.

In [11]:
from qiskit_optimization.problems import QuadraticProgram
qp = QuadraticProgram()
qp.from_docplex(docplex_model)  # deprecated
print(qp.export_as_lp_string())

<ipython-input-11-3f8603b06c58>:3: DeprecationWarning: The from_docplex method is deprecated as of version 0.2.0 and will be removed no sooner than 3 months after the release. Instead use the qiskit_optimization.translators.from_docplex_mp function.
  qp.from_docplex(docplex_model)  # deprecated
\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: docplex

Maximize
 obj: [ 2 x*y ]/2
Subject To
 c0: x - y <= 0

Bounds
 0 <= x <= 1
 -1 <= y <= 4

Binaries
 x

Generals
 y
End

